# Pandas

# Table of Contents
  - [Pandas](#Pandas)
  - [Introduction](#Introduction)
      - [What is Pandas?](#What-is-Pandas?)
      - [Why We Need Pandas with Python](#Why-We-Need-Pandas-with-Python)
      - [When and Why It's Used](#When-and-Why-It's-Used)
      - [Pandas Performance](#Pandas-Performance)
      - [Alternatives and When They Are Better](#Alternatives-and-When-They-Are-Better)
  - [References](#References)
  - [Working with Pandas `DataFrame`](#Working-with-Pandas-DataFrame)
    - [Pandas Data Structures](#Pandas-Data-Structures)
      - [Working with NumPy Arrays](#Working-with-NumPy-Arrays)
      - [`Series`](#Series)
      - [`Index`](#Index)
      - [`DataFrame`](#DataFrame)
    - [Creating `DataFrame` objects](#Creating-DataFrame-objects)
      - [Creating a `Series` object](#Creating-a-Series-object)
      - [Creating a `DataFrame` object from a `Series` object](#Creating-a-DataFrame-object-from-a-Series-object)
      - [Creating a `DataFrame` from Python Data Structures](#Creating-a-DataFrame-from-Python-Data-Structures)
        - [From a dictionary of list-like structures](#From-a-dictionary-of-list-like-structures)
        - [From a list of dictionaries](#From-a-list-of-dictionaries)
        - [From a list of tuples](#From-a-list-of-tuples)
        - [From a NumPy array](#From-a-NumPy-array)
    - [Creating a `DataFrame` object from the contents of a CSV File](#Creating-a-DataFrame-object-from-the-contents-of-a-CSV-File)
      - [Finding information on the file before reading it in](#Finding-information-on-the-file-before-reading-it-in)
        - [Examining a few rows](#Examining-a-few-rows)
        - [Column count](#Column-count)
      - [Reading in the file](#Reading-in-the-file)
      - [Writing a `DataFrame` Object to a CSV File](#Writing-a-DataFrame-Object-to-a-CSV-File)
    - [Writing a `DataFrame` Object to a Database](#Writing-a-DataFrame-Object-to-a-Database)
    - [Creating a `DataFrame` Object by Querying a Database](#Creating-a-DataFrame-Object-by-Querying-a-Database)
    - [Selecting data](#Selecting-data)
      - [Selecting columns](#Selecting-columns)
      - [Slicing](#Slicing)
          - [Selecting rows](#Selecting-rows)
          - [Selecting rows and columns with chaining](#Selecting-rows-and-columns-with-chaining)
      - [Indexing](#Indexing)
        - [Indexing with `loc`](#Indexing-with-loc)
        - [Indexing with `iloc`](#Indexing-with-iloc)
        - [Looking up scalar values](#Looking-up-scalar-values)
      - [Filtering](#Filtering)
    - [Adding and removing data](#Adding-and-removing-data)
      - [Adding new columns](#Adding-new-columns)
      - [Concatenation](#Concatenation)
      - [Deleting data](#Deleting-data)
          - [Using the `drop()` method](#Using-the-drop()-method)
    - [Exercises](#Exercises)
  - [Data wrangling](#Data-wrangling)
    - [Cleaning data](#Cleaning-data)
      - [Renaming Columns](#Renaming-Columns)
      - [Type Conversion](#Type-Conversion)
      - [Reordering, reindexing, and sorting](#Reordering,-reindexing,-and-sorting)
    - [Reshaping data](#Reshaping-data)
      - [Transposing](#Transposing)
      - [Pivoting](#Pivoting)
        - [`pivot()`](#pivot())
        - [`unstack()`](#unstack())
      - [Melting](#Melting)
        - [`melt()`](#melt())
        - [`stack()`](#stack())
    - [Exercises](#Exercises)
  - [Data aggregation](#Data-aggregation)

# Introduction



### What is Pandas?

[Pandas](https://pandas.pydata.org/docs/index.html) is an open-source data manipulation and analysis library for Python.
It provides powerful data structures and functions designed to make working with structured data intuitive and efficient. At the heart of Pandas are two primary data structures:

- **DataFrame**: A two-dimensional, size-mutable, and potentially heterogeneous tabular data structure with labeled axes (rows and columns). It's similar to a spreadsheet or SQL table and is generally the most commonly used Pandas object.
- **Series**: A one-dimensional labeled array capable of holding any data type (integers, strings, floating point numbers, Python objects, etc.).

Pandas integrates well with various other Python libraries, such as Matplotlib for plotting and NumPy for numerical computations, making it a central library in the Python data science stack.

### Why We Need Pandas with Python

Python, while a powerful programming language, isn't designed specifically for data analysis.
It lacks built-in, high-level data structures and tools that are intuitive and efficient for these tasks.
Here's where Pandas comes in:

- **Data Cleaning and Preparation**: Data scientists spend a significant amount of time cleaning and preparing data. Pandas simplifies these tasks with built-in functions for filtering, selecting, and manipulating data.
- **Data Analysis**: With Pandas, analyzing and exploring data is more straightforward. It provides functions for aggregating, summarizing, and transforming data, making it easier to derive insights.
- **Data Visualization**: Though Pandas is not a data visualization library, it seamlessly interfaces with Matplotlib for plotting and visualizing data, allowing quick and informative visual analysis.
- **Handling Diverse Data Types**: Pandas efficiently handles a variety of data formats, including CSV, Excel files, SQL databases, and HDF5 format, making it a versatile tool for diverse data analysis needs.

### When and Why It's Used

Pandas is widely used in a variety of fields for data analysis and manipulation tasks.
Some common use cases include:

- **Data Cleaning**: Transforming raw data into a form that is suitable for analysis, such as filling missing values, removing duplicates, and converting data types.
- **Data Exploration and Analysis**: Quick examination of data for patterns, irregularities, and insights. This includes operations like sorting, filtering, and grouping data.
- **Data Visualization**: Creating plots and graphs to understand trends and patterns in data.
- **Machine Learning**: Preprocessing and cleaning datasets before feeding them into machine learning models.

### Pandas Performance

Pandas is highly efficient for most data manipulation and analysis tasks, especially with small to moderately sized datasets.
It's optimized for performance in many scenarios, with critical code paths written in Cython or C.
However, when working with very large datasets (with about tens of millions of rows or more), Pandas can face performance issues due to:

- **Memory Usage**: Pandas typically requires significantly more memory than the size of the data, making it less efficient for very large datasets.
- **Speed**: For extremely large datasets, some operations in Pandas can be slow, as it's not fully optimized for all use cases, especially those involving large-scale, distributed computing.

### Alternatives and When They Are Better

One of the notable alternatives to Pandas is [**Polars**](https://pola.rs/).
Polars is a DataFrame library that is designed to handle larger datasets more efficiently than Pandas.
Here's why and when Polars can be a better choice:

- **Performance**: Polars is designed to be faster and more memory-efficient than Pandas, particularly with large datasets. It leverages modern hardware capabilities, like multi-threading, to speed up data processing.
- **Lazy Evaluation**: Polars supports lazy evaluation, where computations are queued and executed only when necessary. This approach can lead to performance improvements, especially in complex data pipelines.
- **Ease of Scaling**: For large-scale data processing, Polars can be a better fit. It's more adept at handling the kinds of big data tasks that are increasingly common in industry settings.

---

# References

The majority of the material of this notebook has been taken and rearranged – with the permission of the author – from the book [*Hands-On Data Analysis with Pandas* (2nd edition) by Stefanie Molin](https://www.amazon.com/dp/1800563450/).
The book covers way more than just Pandas, including plotting, data mining, and some guided projects to build "data-centric" apps.
If you want to go more in depth with this topics, the book is a highly encouraged read.

---

# Working with Pandas `DataFrame`

## Pandas Data Structures


In this section, we will discuss the `Series`, `Index`, and `DataFrame` classes. To do so, we will read in a snippet of the CSV file we will work with later. Don't worry about that part yet, though.

### Working with NumPy Arrays
Let's read in a short CSV file (using `numpy`) for some sample data. 

In [ ]:
import numpy as np

data = np.genfromtxt(
    'data/01/example_data.csv', delimiter=';', 
    names=True, dtype=None, encoding='UTF'
)
data

We can find the dimensions with the `shape` attribute:

In [ ]:
data.shape

We can find the data types with the `dtype` attribute:

In [ ]:
data.dtype

Each element in the array corresponds to a row from the CSV file. Unlike lists that can hold multiple data types, NumPy arrays are limited to one, enabling quick, vectorized actions. The data import resulted in an array of `numpy.void` objects, designed to handle various types. This occurs as each row contains diverse data types: four strings, one float, and one integer. Consequently, we miss out on the performance benefits NumPy offers for arrays with uniform data types.

Consider finding the highest magnitude. We can employ a [list comprehension](https://www.python.org/dev/peps/pep-0202/) to extract the third index from each row, which is in the form of a `numpy.void` object. By doing this, we create a list that allows us to determine the maximum value using the `max()` function.

In [ ]:
%%timeit
max([row[3] for row in data])

If we, instead, create a NumPy array for each column, this operation is much easier (and more efficient) to perform. We can use a **[dictionary comprehension](https://www.python.org/dev/peps/pep-0274/)** to make a dictionary where the keys are the column names and the values are NumPy arrays of the data:

In [ ]:
array_dict = {
    col: np.array([row[i] for row in data])
    for i, col in enumerate(data.dtype.names)
}
array_dict

Grabbing the maximum magnitude is now simply a matter of selecting the `mag` key and calling the `max()` method. This is nearly twice as fast as the list comprehension implementation when dealing with just 5 entries, imagine how much worse the first attempt will perform on large data sets:

In [ ]:
%%timeit
array_dict['mag'].max()

However, this representation has other issues. Say we wanted to grab all the information for the earthquake with the maximum magnitude, how would we go about that? We would need to find the index of the maximum and then for each of the keys in the dictionary grab that index:

In [ ]:
np.array([
    value[array_dict['mag'].argmax()] 
    for key, value in array_dict.items()
])

We now have a NumPy array consisting solely of strings, converting our numerical values into this format and reverting to the earlier setup. Additionally, if we aim to sort the data by magnitude in ascending order, the initial format requires sorting the rows based on the third index. In the second format, we need to establish the sorting order based on the `mag` column and then rearrange all other arrays accordingly. Handling multiple NumPy arrays with different data types simultaneously can be challenging. This is where `pandas` comes into play, enhancing the ease of working with NumPy arrays. Let's begin delving into `pandas` by understanding its data structure.

### `Series`
The `Series` class provides a data structure for arrays of a single type with some additional functionality.

In [ ]:
import pandas as pd

place = pd.Series(array_dict['place'], name='place')
place

Here are some commonly used attributes with `Series` objects:

|Attribute | Returns |
| --- | --- |
| `name` | The name of the `Series` object |
| `dtype` | The data type of the `Series` object |
| `shape` | Dimensions of the `Series` object in a tuple of the form `(number of rows,)` |
| `index` | The `Index` object that is part of the `Series` object |
| `values` | The data in the `Series` object |

For the most part, `pandas` objects use NumPy arrays for their internal data representations. However, for some data types, `pandas` builds upon NumPy to create its own [arrays](https://pandas.pydata.org/pandas-docs/stable/reference/arrays.html). For this reason, depending on the data type, `values` can either be a `pandas.array` or `numpy.array` object. Therefore, if we need to ensure we get a specific type back, then it is recommended to use the `array` attribute or `to_numpy()` method, respectively, instead of `values`.

Now let's see some examples using these attributes.

**Getting the name of the series**

The NumPy array held the name of the data in the `dtype` attribute; here, we can access it directly: 

In [ ]:
place.name

**Getting the data type**

A `Series` object holds a single data type.
Here it is `'O'` for object.

In [ ]:
place.dtype

**Getting the dimensions of the series**

Just as with NumPy, we can use `shape` to get the dimensions as `(rows, columns)`.
`Series` objects are a single column, so they only have values for the rows dimension. 

In [ ]:
place.shape

**Isolating the values from the series**

This `Series` object is storing its values as a NumPy array:

In [ ]:
place.values

### `Index`
The addition of the `Index` class makes the `Series` class more powerful than a NumPy array. We can get the index from the `index` attribute of a `Series` object:

In [ ]:
place_index = place.index
place_index

As with `Series` objects, we can access the underlying data via the `values` attribute. Note that this `Index` object is also built on top of a NumPy array:

In [ ]:
place_index.values

Here are some commonly used attributes with `Index` objects:

|Attribute | Returns |
| --- | --- |
| `name` | The name of the `Index` object |
| `dtype` | The data type of the `Index` object |
| `shape` | Dimensions of the `Index` object |
| `values` | The data in the `Index` object |
| `is_unique` | Check if the `Index` object has all unique values |

We can check the type of the underlying data, just like with a `Series` object:

In [ ]:
place_index.dtype

Same for the dimensions:

In [ ]:
place_index.shape

We can check if the values are unique:

In [ ]:
place_index.is_unique

With NumPy we can perform arithmetic operations element-wise between arrays:

In [ ]:
np.array([1, 1, 1]) + np.array([-1, 0, 1])

Pandas supports this as well, and the index determines how element-wise operations are performed. With addition, only the matching indices are summed:

In [ ]:
numbers = np.linspace(0, 10, num=5) # makes numpy array([0, 2.5, 5, 7.5, 10])
x = pd.Series(numbers) # index is [0, 1, 2, 3, 4]
y = pd.Series(numbers, index=pd.Index([1, 2, 3, 4, 5]))
x + y

We aren't limited to the integer indices of list-like structures, and we can label our rows. The labels can be altered at any time and be things like dates or even another column. In chapter 3, we will discuss how to perform some operations on the index in order to change it. Then, in chapter 4, we will use the index for operations merging data and aggregating it.



### `DataFrame`

Using a `Series` object for each column enhances the NumPy approach, yet challenges persist in sorting by values or extracting full rows.
A `DataFrame` provides a tabular representation comprising multiple `Series` objects as columns and a unified `Index` object labeling the rows.
We can construct a `DataFrame` from either of the previously discussed NumPy formats.
While it's possible to create a `Series` object for each column, it's unnecessary:

In [ ]:
df = pd.DataFrame(array_dict) 

# this will also work with the first representation
# df = pd.DataFrame(data)

df

We can check the type of the underlying data with `dtypes` (note that it is not `dtype` as with `Series` and `Index` objects since each column will have its own data type):

In [ ]:
df.dtypes

We can get the underlying data with the `values` attribute. Note that this looks very similar to our initial NumPy representation:

In [ ]:
df.values

We can isolate the columns with the `columns` attribute. Notice that the columns are actually an `Index` object just on a different axis (columns are the horizontal index while rows are the vertical index).

In [ ]:
df.columns

Here are some commonly used attributes:

|Attribute | Returns |
| --- | --- |
| `dtypes` | The data types of each column |
| `shape` | Dimensions of the `DataFrame` object in a tuple of the form `(number of rows, number of columns)` |
| `index` | The `Index` object along the rows of the `DataFrame` object |
| `columns` | The name of the columns (as an `Index` object) |
| `values` | The data in the `DataFrame` object |
| `empty` | Check if the `DataFrame` object is empty |

The `Index` object along the rows of the dataframe can be accessed via the `index` attribute (just as with `Series` objects):

In [ ]:
df.index

As with both `Series` and `Index` objects, we can get the dimensions of the dataframe with the `shape` attribute. The result is of the form `(nrows, ncols)`. Our dataframe has 5 rows and 6 columns:

In [ ]:
df.shape

Pandas allows arithmetic operations on dataframes, matching both index and column for execution.
This example showcases addition.
In string columns (`time`, `place`, `magType`, and `alert`), pandas concatenates values across dataframes.
For numeric columns (`mag` and `tsunami`), the values are summed.

In [ ]:
df + df

## Creating `DataFrame` objects

In [ ]:
import datetime as dt
import numpy as np
import pandas as pd

### Creating a `Series` object

In [ ]:
np.random.seed(0) # set a seed for reproducibility
pd.Series(np.random.rand(5), name='random')

### Creating a `DataFrame` object from a `Series` object
Use the `to_frame()` method:

In [ ]:
pd.Series(np.linspace(0, 10, num=5)).to_frame()

### Creating a `DataFrame` from Python Data Structures

#### From a dictionary of list-like structures

The dictionary values can be lists, NumPy arrays, etc. as long as they have length (generators don't have length so we can't use them here):

In [ ]:
np.random.seed(0) # set seed so result is reproducible
pd.DataFrame(
    {
        'random': np.random.rand(5),
        'text': ['hot', 'warm', 'cool', 'cold', None],
        'truth': [np.random.choice([True, False]) for _ in range(5)]
    }, 
    index=pd.date_range(
        end=dt.date(2019, 4, 21),
        freq='1D',
        periods=5, 
        name='date'
    )
)

#### From a list of dictionaries

In [ ]:
pd.DataFrame([
    {'mag': 5.2, 'place': 'California'},
    {'mag': 1.2, 'place': 'Alaska'},
    {'mag': 0.2, 'place': 'California'},
])

#### From a list of tuples

In [ ]:
list_of_tuples = [(n, n**2, n**3) for n in range(5)]
list_of_tuples

In [ ]:
pd.DataFrame(
    list_of_tuples, 
    columns=['n', 'n_squared', 'n_cubed']
)

#### From a NumPy array

In [ ]:
pd.DataFrame(
    np.array([
        [0, 0, 0],
        [1, 1, 1],
        [2, 4, 8],
        [3, 9, 27],
        [4, 16, 64]
    ]), columns=['n', 'n_squared', 'n_cubed']
)

## Creating a `DataFrame` object from the contents of a CSV File

### Finding information on the file before reading it in
Before attempting to read in a file, we can use the command line to see important information about the file that may determine how we read it in. We can run command line code from Jupyter Notebooks by using `!` before the code.

For example, we can find out how many lines are in the file by using the `wc` utility (word count) and counting lines in the file (`-l`). The file has 9,333 lines:

In [ ]:
!wc -l data/earthquakes.csv

**Windows users**: if the above doesn't work for you (depends on your setup), then use this instead:

```python
!find /c /v "" data\earthquakes.csv
```


We can find the file size by using `ls` to list the files in the `data` directory, and passing in the flags `-lh` to include the file size in human readable format. Then we use `grep` to find the file in question. Note that `|` passes the result of `ls` to `grep`. The `grep` utility is used for finding items that match patterns.

This tells us the file is 3.4 MB:

In [ ]:
!ls -lh data | grep earthquakes.csv

**Windows users**: if the above doesn't work for you (depends on your setup), then use this instead:

```python
!dir data | findstr "earthquakes.csv"
```

We can even capture the result of a command and use it in our Python code:

In [ ]:
files = !ls -lh data
[file for file in files if 'earthquake' in file]

**Windows users**: if the above doesn't work for you (depends on your setup), then use this instead:

```python
files = !dir data
[file for file in files if 'earthquake' in file]
```

#### Examining a few rows

We can use `head` to look at the top `n` rows of the file. With the `-n` flag, we can specify how many. This shows use that the first row of the file contains headers and that it is comma-separated (just because the file extension is `.csv` doesn't it contains comma-separated values):

In [ ]:
!head -n 2 data/earthquakes.csv

**Windows users**: if the above doesn't work for you (depends on your setup), then use this instead:

```python
n = 2
with open('data/earthquakes.csv', 'r') as file:
    for _ in range(n):
        print(file.readline(), end='\r')
```


Just like `head` gives rows from the top, `tail` gives rows from the bottom. This can help us check that there is no extraneous data on the bottom of the field, like perhaps some metadata about the fields that actually isn't part of the dataset:

In [ ]:
!tail -n 1 data/earthquakes.csv

**Windows users**: if the above doesn't work for you (depends on your setup), then use this instead:

```python
import os

with open('data/earthquakes.csv', 'rb') as file:
    file.seek(0, os.SEEK_END)
    while file.read(1) != b'\n':
        file.seek(-2, os.SEEK_CUR)
    print(file.readline().decode())
```

*Note*: To inspect more than one row from the end of the file, you will have to use this instead, which requires reading the whole file:

```python
n = 2
with open('data/earthquakes.csv', 'r') as file:
    print('\r'.join(file.readlines()[-n:]))
```



#### Column count
We can use `awk` to find the column count. This is a utility for pattern scanning and processing. The `-F` flag allows us to specify the delimiter (comma, in this case). Then we specify what to do for each record in the file. We choose to print `NF` which is a predefined variable whose value is the number of fields in the current record. Here, we say `exit` so that we print the number of fields (columns, here) in the first row of the file, then we stop. 

This tells us we have 26 data columns:

In [ ]:
!awk -F',' '{print NF; exit}' data/earthquakes.csv

**Windows users**: if the above or below don't work for you (depends on your setup), then use this instead:

```python
with open('data/earthquakes.csv', 'r') as file:
    print(len(file.readline().split(',')))
```


Since we know the 1st line of the file had headers, and the file is comma-separated, we can also count the columns by using `head` to get headers and parsing them in Python:

In [ ]:
headers = !head -n 1 data/earthquakes.csv
len(headers[0].split(','))

**Windows users**: if you had to use the alternatives above, consider trying out [Cygwin](https://www.cygwin.com) or [Windows Subsystem for Linux (WSL)](https://docs.microsoft.com/en-us/windows/wsl/about).



### Reading in the file

Our file is small in size, has headers in the first row, and is comma-separated, so we don't need to provide any additional arguments to read in the file with `pd.read_csv()`, but be sure to check the [documentation](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) for possible arguments:

In [ ]:
df = pd.read_csv('data/earthquakes.csv')

Note that we can also pass in a URL. Let's read this same file from GitHub:

In [ ]:
df = pd.read_csv(
    'https://github.com/stefmolin/'
    'Hands-On-Data-Analysis-with-Pandas-2nd-edition'
    '/blob/master/ch_02/data/earthquakes.csv?raw=True'
)

Pandas is usually very good at figuring out which options to use based on the input data, so we often won't need to add arguments to the call; however, there are many options available should we need them, some of which include the following:

| Parameter | Purpose |
| --- | --- |
| `sep` | Specifies the delimiter |
| `header` | Row number where the column names are located; the default option has `pandas` infer whether they are present |
| `names` | List of column names to use as the header |
| `index_col` | Column to use as the index |
| `usecols` | Specifies which columns to read in |
| `dtype` | Specifies data types for the columns | 
| `converters` | Specifies functions for converting data in certain columns |
| `skiprows` | Rows to skip |
| `nrows` | Number of rows to read at a time (combine with `skiprows` to read a file bit by bit) |
| `parse_dates` | Automatically parse columns containing dates into datetime objects |
| `chunksize` | For reading the file in chunks |
| `compression` | For reading in compressed files without extracting beforehand |
| `encoding` | Specifies the file encoding |

### Writing a `DataFrame` Object to a CSV File

Our file is small in size, has headers in the first row, and is comma-separated, so we don't need to provide any additional arguments to read in the file with `pd.read_csv()`, but be sure to check the [documentation](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) for possible arguments:

In [ ]:
df.to_csv('output.csv', index=False)

## Writing a `DataFrame` Object to a Database
Note the `if_exists` parameter. By default, it will give you an error if you try to write a table that already exists. Here, we don't care if it is overwritten. Lastly, if we are interested in appending new rows, we set that to `'append'`.

In [ ]:
import sqlite3

with sqlite3.connect('data/01/quakes.db') as connection:
    pd.read_csv('data/01/tsunamis.csv').to_sql(
        'tsunamis', connection, index=False, if_exists='replace'
    )

## Creating a `DataFrame` Object by Querying a Database
Using a SQLite database. Otherwise you need to install [SQLAlchemy](https://www.sqlalchemy.org/).

In [ ]:
import sqlite3

with sqlite3.connect('data/01/quakes.db') as connection:
    tsunamis = pd.read_sql('SELECT * FROM tsunamis', connection)

tsunamis.head()

## Selecting data

We are going to use the `earthquakes.csv` file again, so let's read it in as a `DataFrame`:

In [ ]:
import pandas as pd

df = pd.read_csv('data/01/earthquakes.csv')

### Selecting columns

In [ ]:
df.mag

Grab an entire column using dictionary syntax:

In [ ]:
df['mag']

Selecting multiple columns:

In [ ]:
df[['mag', 'title']]

Selecting columns using list comprehensions and string operations:

In [ ]:
df[
    ['title', 'time']
    + [col for col in df.columns if col.startswith('mag')]
]

Breaking down this example:
1. the list comprehension

In [ ]:
[col for col in df.columns if col.startswith('mag')]

2. assembling the list

In [ ]:
['title', 'time'] \
+ [col for col in df.columns if col.startswith('mag')]

3. using this list as the list of columns

In [ ]:
df[
    ['title', 'time']
    + [col for col in df.columns if col.startswith('mag')]
]

### Slicing

##### Selecting rows

We can use the standard Python sytnax for slicing to fetch a number of rows:

In [ ]:
df[100:103]

##### Selecting rows and columns with chaining

In [ ]:
df[['title', 'time']][100:103]

Order doesn't matter here:

In [ ]:
df[100:103][['title', 'time']].equals(
    df[['title', 'time']][100:103]
)

So we know how to select rows and columns, but can we update values? Well, if we try using what we have learned so far, we will see the following warning:

In [ ]:
df[110:113]['title'] = df[110:113]['title'].str.lower()

Note that it worked here, but `pandas` says we were setting a value on a copy of a slice and that we should use `loc` instead (see the following section):

In [ ]:
df[110:113]['title']

### Indexing

We can use the `loc` method of a `DataFrame` to modify a slice of a data frame without incurring in Pandas warnings:

In [ ]:
df.loc[110:112, 'title'] = df.loc[110:112, 'title'].str.lower()
df.loc[110:112, 'title']

#### Indexing with `loc`

Selection of the format `loc[row_indexer, column_indexer]` where `:` can be used to select all, as in Python's slicing syntax:

In [ ]:
df.loc[:,'title']

We can use `loc` to select specific rows and columns without chaining. If we use row numbers with `loc`, they are now **inclusive** of the end index:

In [ ]:
df.loc[10:15, ['title', 'mag']]

#### Indexing with `iloc`
Exclusive of the endpoint just as Python slicing:

In [ ]:
df.iloc[10:15, [19, 8]]

We can use slicing syntax with `iloc` for both rows and columns:

In [ ]:
df.iloc[10:15, 6:10]

When using `loc`, we can slice on column names. This will be inclusive of the endpoint because you can't be expected to know what the next column name will be. As such, we have multiple ways to achieve the same end goal:

In [ ]:
df.iloc[10:15, 6:10].equals(
    df.loc[10:14, 'gap':'magType']
)

#### Looking up scalar values
We used `loc` and `iloc` to grab subsets of the dataframe. However, if we are just interested in the specific value at a given `[row, column]`, then we can use `iat` and `at`. We use `at` with labels:

In [ ]:
df.at[10, 'mag']

...and `iat` with integer indices:

In [ ]:
df.iat[10, 8]

### Filtering

Similar to NumPy arrays, we can use **Boolean mask** to filter a data frame:

In [ ]:
df.mag > 2

This returns a "list" of every row for which the criteria on the `mag` column is greater than `2`.
If we then want to select our data, we simply place it inside the brackets:

In [ ]:
df[df.mag >= 7.0]

We can use masks with `loc`:

In [ ]:
df.loc[
    df.mag >= 7.0,
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

Masks can be created using multiple criteria when combined with bitwise operators:
- `&` for AND
- `|` for OR

We must also surround each criterion with parentheses. We can't use `and`/`or` here because we need to evaluate row by row:

In [ ]:
df.loc[
    (df.tsunami == 1) & (df.alert == 'red'),
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

An example with an OR condition, which is less restrictive:

In [ ]:
df.loc[
    (df.tsunami == 1) | (df.alert == 'red'),
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

Masks can be created from any criteria that results in a Boolean. For example, we can select all earthquakes with the string `Alaska` in the `place` column with a non-null value for the `alert` column. To get non-nulls, we can use the `isnull()` method with the bitwise negation operator (`~`) or the `notnull()` method:

In [ ]:
df.loc[
    (df.place.str.contains('Alaska')) & (df.alert.notnull()),
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

We can even use regular expressions here:

In [ ]:
df.loc[
    (df.place.str.contains(r'CA|California$')) & (df.mag > 3.8),
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

We can use the `between()` method to turn 2 individual checks (is less than or equal to some maximum value and is greater than or equal to some minimum value) into a single one. Note this is inclusive of the endpoint by default:

In [ ]:
df.loc[
    df.mag.between(6.5, 7.5),
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

We can use the `isin()` method to check for membership in a list of values:

In [ ]:
df.loc[
    df.magType.isin(['mw', 'mwb']),
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

We can grab the index of the minimum and maximum values of a given column and use those to select the entire row where they occur:

In [ ]:
[df.mag.idxmin(), df.mag.idxmax()]

In [ ]:
df.loc[
    [df.mag.idxmin(), df.mag.idxmax()],
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

Note that there is a `filter()` method, but it doesn't filter the data in the same sense as we discussed in this section. Here are a few things you can do with this method.

- grab columns of a dataframe by passing a list to `items`:

In [ ]:
df.filter(items=['mag', 'magType']).head()

- grab all the columns that contain a string with the `like` parameter:

In [ ]:
df.filter(like='mag').head()

- use regular expressions; here, we select any columns that start with `t`:

In [ ]:
df.filter(regex=r'^t').head()

- use `filter()` along the rows, by passing in `axis=0`. Here, we will use the `place` column as the index (we will cover `set_index()` in chapter 3):

In [ ]:
df.set_index('place').filter(like='Japan', axis=0).filter(items=['mag', 'magType', 'title']).head()

This also works on `Series` objects and will run on the index:

In [ ]:
df.set_index('place').title.filter(like='Japan').head()

## Adding and removing data

### Adding new columns

In [ ]:
df['source'] = 'USGS API'
df.head()

...or a Boolean mask:

In [ ]:
df['mag_negative'] = df.mag < 0
df.head()

### Concatenation
Say we were working with two separate dataframes, one with earthquakes accompanied by tsunamis and the other with earthquakes without tsunamis.
If we wanted to look at earthquakes as a whole, we would want to concatenate the dataframes into a single one:

In [ ]:
tsunami = df[df.tsunami == 1]
no_tsunami = df[df.tsunami == 0]

tsunami.shape, no_tsunami.shape

Concatenating along the row axis (`axis=0`) is equivalent to appending to the bottom.
By concatenating our earthquakes with tsunamis and those without tsunamis, we get the full earthquake data set back:

In [ ]:
pd.concat([tsunami, no_tsunami]).shape

Note that the previous result is equivalent to running the `append()` method of the dataframe:

In [ ]:
tsunami.append(no_tsunami).shape

We have been working with a subset of the columns from the CSV file, but suppose that now we want to get some of the columns we ignored when we read in the data. Since we have added new columns in this notebook, we won't want to read in the file and perform those operations again. Instead, we will concatenate along the columns (`axis=1`) to add back what we are missing:

In [ ]:
additional_columns = pd.read_csv(
    'data/earthquakes.csv', usecols=['tz', 'felt', 'ids']
)
pd.concat([df.head(2), additional_columns.head(2)], axis=1)

Notice what happens if the index doesn't align though:

In [ ]:
additional_columns = pd.read_csv(
    'data/earthquakes.csv', usecols=['tz', 'felt', 'ids', 'time'], index_col='time'
)
pd.concat([df.head(2), additional_columns.head(2)], axis=1)

If the index doesn't align, we can align it before attempting the concatentation, which we will discuss in chapter 3.

Say we want to join the `tsunami` and `no_tsunami` dataframes, but the `no_tsunami` dataframe has an additional column. The `join` parameter specifies how to handle any overlap in column names (when appending to the bottom) or in row names (when concatenating to the left/right). By default, this is `outer`, so we keep everything; however, if we use `inner`, we will only keep what is in common:

In [ ]:
pd.concat(
    [tsunami.head(2), no_tsunami.head(2).assign(type='earthquake')], join='inner'
)

In addition, we use `ignore_index`, since the index doesn't mean anything for us here. This gives us sequential values instead of what we had in the previous result:

In [ ]:
pd.concat(
    [tsunami.head(2), no_tsunami.head(2).assign(type='earthquake')], join='inner', ignore_index=True
)

### Deleting data

In [ ]:
del df['source']
df.columns

If we don't know whether the column exists, we should use a `try`/`except` block:

In [ ]:
try:
    del df['source']
except KeyError:
    # handle the error here
    print('not there anymore')

We can also use `pop()`. This will allow us to use the series we remove later. Note there will be an error if the key doesn't exist, so we can also use a `try`/`except` here:

In [ ]:
mag_negative = df.pop('mag_negative')
df.columns

Notice we have a mask in `mag_negative` now:

In [ ]:
mag_negative.value_counts()

Now, we can use `mag_negative` to filter our data:

In [ ]:
df[mag_negative].head()

##### Using the `drop()` method

We can drop rows by passing a list of indices to the `drop()` method. Notice in the following example that when asking for the first 2 rows with `head()` we get the 3rd and 4th rows because we dropped the original first 2 with `drop([0, 1])`:

In [ ]:
df.drop([0, 1]).head(2)

The `drop()` method drops along the row axis by default. If we pass in a list of columns with the `columns` argument, we can delete columns:

In [ ]:
cols_to_drop = [
    col for col in df.columns
    if col not in ['alert', 'mag', 'title', 'time', 'tsunami']
]
df.drop(columns=cols_to_drop).head()

We also have the option of using `axis=1`:

In [ ]:
df.drop(columns=cols_to_drop).equals(
    df.drop(cols_to_drop, axis=1)
)

By default, `drop()`, along with the majority of `DataFrame` methods, will return a new `DataFrame` object. If we just want to change the one we are working with, we can pass `inplace=True`. This should be used with care:

In [ ]:
df.drop(columns=cols_to_drop, inplace=True)
df.head()

## Exercises

Using the CSV file found in `data/01/parsed.csv`:

In [ ]:
%reload_ext tutorial.tests.testsuite

In [ ]:
import pandas as pd

1. Determine the 95th percentile for earthquake magnitudes in Japan, specifically using the mb magnitude scale.


In [ ]:
%%ipytest

def solution_pandas_1(df: pd.DataFrame):
    pass

2. Calculate the proportion of earthquakes in Indonesia that were accompanied by tsunamis.


In [ ]:
%%ipytest

def solution_pandas_2(df: pd.DataFrame):
    pass

# Data wrangling

In this section, we will be using two different datasets:

1. Daily temperature data from the National Centers for Environmental Information (NCEI) API. We will use the Global Historical Climatology Network - Daily (GHCND) dataset.
2. S&P 500 stock market data and data for Bitcoin for 2017 through 2018.


In [ ]:
import pandas as pd

df = pd.read_csv('data/02/nyc_temperatures.csv')
df.head()

## Cleaning data

### Renaming Columns



In [ ]:
df.columns

We want to rename the `value` column to indicate it contains the temperature in Celsius and the `attributes` column to say `flags` since each value in the comma-delimited string is a different flag about the data collection. For this task, we use the `rename()` method and pass in a dictionary mapping the column names to their new names. We pass `inplace=True` to change our original dataframe instead of getting a new one back:

In [ ]:
df.rename(
    columns={
        'value': 'temp_C',
        'attributes': 'flags'
    }, inplace=True
)

Those columns have been successfully renamed:

In [ ]:
df.columns

We can also perform string operations on the column names with `rename()`:

In [ ]:
df.rename(str.upper, axis='columns').columns

### Type Conversion

We want to store the `date` column as a `datetime` object:

In [ ]:
df.dtypes

Let's perform the conversion with `pd.to_datetime()`:

In [ ]:
df.loc[:,'date'] = pd.to_datetime(df.date)
df.dtypes

Now we get useful information when we use `describe()` on this column:

In [ ]:
df.date.describe(datetime_is_numeric=True)

We can use `tz_localize()` on a `DatetimeIndex` object to convert to a desired timezone:

In [ ]:
pd.date_range(start='2018-10-25', periods=2, freq='D').tz_localize('EST')

This also works with `Series`/`DataFrame` objects that have an index of type `DatetimeIndex`. Let's read in the CSV again for this example and set the `date` column to be the index and stored as a datetime:

In [ ]:
eastern = pd.read_csv(
    'data/02/nyc_temperatures.csv', index_col='date', parse_dates=True
).tz_localize('EST')

eastern.head()

We can use `tz_convert()` to convert to another timezone from there. If we convert the Eastern datetimes to UTC, they will now be at 5 AM, since `pandas` will use the offsets to convert:

In [ ]:
eastern.tz_convert('UTC').head()

We can change the period of the index as well. We could change the period to be monthly to make it easier to aggregate later. (Aggregation will be discussed in chapter 4.)

In [ ]:
eastern.tz_localize(None).to_period('M').index

We now get a `PeriodIndex` object, which we can change back into a `DatetimeIndex` object with `to_timestamp()`:

In [ ]:
eastern.tz_localize(None).to_period('M').to_timestamp().index

We can use the `assign()` method for working with multiple columns at once (or creating new ones). Since our `date` column has already been converted, we need to read in the data again:

In [ ]:
df = pd.read_csv('data/02/nyc_temperatures.csv').rename(
    columns={
        'value': 'temp_C',
        'attributes': 'flags'
    }
)

new_df = df.assign(
    date=pd.to_datetime(df.date),
    temp_F=(df.temp_C * 9/5) + 32
)

new_df.dtypes

The `date` column now has datetimes and the `temp_F` column was added:

In [ ]:
new_df.head()

We can also use `astype()` to perform conversions. Let's create columns of the integer portion of the temperatures in Celsius and Fahrenheit. We will use **lambda functions** (first introduced in *Chapter 2, Working with Pandas DataFrames*), so that we can use the values being created in the `temp_F` column to calculate the `temp_F_whole` column. It is very common (and useful) to use lambda functions with `assign()`:

In [ ]:
df = df.assign(
    date=lambda x: pd.to_datetime(x.date),
    temp_C_whole=lambda x: x.temp_C.astype('int'),
    temp_F=lambda x: (x.temp_C * 9/5) + 32,
    temp_F_whole=lambda x: x.temp_F.astype('int')
)

df.head()

Creating categories:

In [ ]:
df_with_categories = df.assign(
    station=df.station.astype('category'),
    datatype=df.datatype.astype('category')
)
df_with_categories.dtypes

In [ ]:
df_with_categories.describe(include='category')

Our categories have no order, but this is something that `pandas` supports:

In [ ]:
pd.Categorical(
    ['med', 'med', 'low', 'high'], 
    categories=['low', 'med', 'high'],
    ordered=True
)

### Reordering, reindexing, and sorting
Say we want to find the days that reached the hottest temperatures in the weather data; we can sort our values by the `temp_C` column with the largest on top to find this: 

In [ ]:
df[df.datatype == 'TMAX'].sort_values(by='temp_C', ascending=False).head(10)

However, this isn't perfect because we have some ties, and they aren't sorted consistently.
In the first tie between the 7th and the 10th, the earlier date comes first, but the opposite is true with the tie between the 4th and the 2nd.
We can use other columns to break ties and specify how to sort each with `ascending`.
Let's break ties with the date column and show earlier dates before later ones:

In [ ]:
df[df.datatype == 'TMAX'].sort_values(by=['temp_C', 'date'], ascending=[False, True]).head(10)

Notice that the index was jumbled in the past 2 results. Here, our index only stores the row number in the original data, but we may not need to keep track of that information. In this case, we can pass in `ignore_index=True` to get a new index after sorting:

In [ ]:
df[df.datatype == 'TMAX'].sort_values(by=['temp_C', 'date'], ascending=[False, True], ignore_index=True).head(10)

When just looking for the n-largest values, rather than wanting to sort all the data, we can use `nlargest()`:

In [ ]:
df[df.datatype == 'TAVG'].nlargest(n=10, columns='temp_C')

We use `nsmallest()` for the n-smallest values.

In [ ]:
df.nsmallest(n=5, columns=['temp_C', 'date'])

The `sample()` method will give us rows (or columns with `axis=1`) at random.
We can provide a seed (`random_state`) to make this reproducible.
The index after we do this is jumbled:

In [ ]:
df.sample(5, random_state=0).index

We can use `sort_index()` to order it again:

In [ ]:
df.sample(5, random_state=0).sort_index().index

The `sort_index()` method can also sort columns alphabetically:

In [ ]:
df.sort_index(axis=1).head()

This can make selection with `loc` easier for many columns:

In [ ]:
df.sort_index(axis=1).head().loc[:,'temp_C':'temp_F_whole']

We must sort the index to compare two dataframes. If the index is different, but the data is the same, they will be marked not-equal:

In [ ]:
df.equals(df.sort_values(by='temp_C'))

Sorting the index solves this issue:

In [ ]:
df.equals(df.sort_values(by='temp_C').sort_index())

Let's set the `date` column as our index:

In [ ]:
df.set_index('date', inplace=True)
df.head()

Now that we have an index of type `DatetimeIndex`, we can do datetime slicing and indexing.
As long as we provide a date format that pandas understands, we can grab the data.
To select all of 2018, we simply use `df.loc['2018']`, for the fourth quarter of 2018 we can use `df.loc['2018-Q4']`, grabbing October is as simple as using `df.loc['2018-10']`; these can also be combined to build ranges.

Let's grab October 11, 2018 through October 12, 2018 (inclusive of both endpoints)&mdash;note that using `loc[]` is optional for ranges:

In [ ]:
df['2018-10-11':'2018-10-12']

We can also use `reset_index()` to get a fresh index and move our current index into a column for safe keeping. This is especially useful if we had data, such as the date, in the index that we don't want to lose:

In [ ]:
df['2018-10-11':'2018-10-12'].reset_index()

Reindexing allows us to conform our axis to contain a given set of labels.

Let's turn to the S&P 500 stock data in the `sp500.csv` file to see an example of this.
Notice we only have data for trading days (weekdays, excluding holidays):

In [ ]:
sp = pd.read_csv(
    'data/02/sp500.csv', index_col='date', parse_dates=True
).drop(columns=['adj_close'])

sp.head(10).assign(
    day_of_week=lambda x: x.index.day_name()
)

If we want to look at the value of a portfolio (group of assets) that trade on different days, we need to handle the mismatch in the index.
Bitcoin, for example, trades daily.
If we sum up all the data we have for each day (aggregations will be covered in chapter 4, so don't fixate on this part), we get the following:

In [ ]:
bitcoin = pd.read_csv(
    'data/02/bitcoin.csv', index_col='date', parse_dates=True
).drop(columns=['market_cap'])

# every day's closing price = S&P 500 close + Bitcoin close (same for other metrics)
portfolio = pd.concat([sp, bitcoin], sort=False).groupby(level='date').sum()

portfolio.head(10).assign(
    day_of_week=lambda x: x.index.day_name()
)

It may not be immediately obvious what is wrong with the previous data, but with a visualization we can easily see the cyclical pattern of drops on the days the stock market is closed.

We will need to import `matplotlib` now:

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter

Now we can see why we need to reindex:

In [ ]:
# plot the closing price from Q4 2017 through Q2 2018
ax = portfolio['2017-Q4':'2018-Q2'].plot(
    y='close', figsize=(15, 5), legend=False,
    title='Bitcoin + S&P 500 value without accounting for different indices'
)

# formatting
ax.set_ylabel('price')
ax.yaxis.set_major_formatter(StrMethodFormatter('${x:,.0f}'))
for spine in ['top', 'right']:
    ax.spines[spine].set_visible(False)

# show the plot
plt.show()

We need to align the index of the S&P 500 to match bitcoin in order to fix this.
We will use the `reindex()` method, but by default we get `NaN` for the values that we don't have data for:

In [ ]:
sp.reindex(bitcoin.index).head(10).assign(
    day_of_week=lambda x: x.index.day_name()
)

So now we have rows for every day of the year, but all the weekends and holidays have `NaN` values.
To address this, we can specify how to handle missing values with the `method` argument.
In this case, we want to forward-fill, which will put the weekend and holiday values as the value they had for the Friday (or end of trading week) before:

In [ ]:
sp.reindex(bitcoin.index, method='ffill').head(10)
    .assign(day_of_week=lambda x: x.index.day_name())

To isolate the changes happening with the forward-filling, we can use the `compare()` method.
It shows us the values that differ across identically-labeled dataframes (same names and same columns).
Here, we can see that only weekends and holidays (Monday, January 16, 2017 was MLK day) have values forward-filled.
Consecutive days have the same values.

In [ ]:
sp.reindex(bitcoin.index)
    .compare(sp.reindex(bitcoin.index, method='ffill'))
    .head(10).assign(day_of_week=lambda x: x.index.day_name())

This isn't perfect though.
We probably want 0 for the volume traded and to put the closing price for the open, high, low, and close on the days the market is closed:

In [ ]:
import numpy as np

sp_reindexed = sp.reindex(bitcoin.index).assign(
    volume=lambda x: x.volume.ffill(0), # put 0 when market is closed
    close=lambda x: x.close.ffill(), # carry this forward
    # take the closing price if these aren't available
    open=lambda x: np.where(x.open.isnull(), x.close, x.open),
    high=lambda x: np.where(x.high.isnull(), x.close, x.high),
    low=lambda x: np.where(x.low.isnull(), x.close, x.low)
)

sp_reindexed.head(10).assign(
    day_of_week=lambda x: x.index.day_name()
)

If we create a visualization comparing the reindexed data to the first attempt, we see how reindexing helped maintain the asset value when the market was closed:

In [ ]:
# every day's closing price = S&P 500 close adjusted for market closure + Bitcoin close (same for other metrics)
fixed_portfolio = sp_reindexed + bitcoin

# plot the reindexed portfolio's closing price from Q4 2017 through Q2 2018
ax = fixed_portfolio['2017-Q4':'2018-Q2'].plot(
    y='close', label='reindexed portfolio of S&P 500 + Bitcoin', figsize=(15, 5), linewidth=2, 
    title='Reindexed portfolio vs. portfolio with mismatched indices'
)

# add line for original portfolio for comparison
portfolio['2017-Q4':'2018-Q2'].plot(
    y='close', ax=ax, linestyle='--', label='portfolio of S&P 500 + Bitcoin w/o reindexing'
)

# formatting
ax.set_ylabel('price')
ax.yaxis.set_major_formatter(StrMethodFormatter('${x:,.0f}'))
for spine in ['top', 'right']:
    ax.spines[spine].set_visible(False)

# show the plot
plt.show()

## Reshaping data

Let's import some data to showcase a few examples of reshaping:

In [ ]:
import pandas as pd

long_df = pd.read_csv(
    'data/02/long_data.csv', usecols=['date', 'datatype', 'value']
).rename(
    columns={'value': 'temp_C'}
).assign(
    date=lambda x: pd.to_datetime(x.date),
    temp_F=lambda x: (x.temp_C * 9/5) + 32
)

long_df.head()

### Transposing
Transposing swaps the rows and the columns. We use the `T` attribute to do so:

In [ ]:
long_df.set_index('date').head(6).T

### Pivoting

#### `pivot()`
We can restructure our data by picking a column to go in the index (`index`), a column whose unique values will become column names (`columns`), and the values to place in those columns (`values`).
The `pivot()` method can be used when we don't need to perform any aggregation in addition to our restructuring (when our index is unique); if this is not the case, we need the `pivot_table()` method which we will cover in the [next chapter](#Data-aggregation). 

In [ ]:
pivoted_df = long_df.pivot(
    index='date', columns='datatype', values='temp_C'
)
pivoted_df.head()

Now that the data is pivoted, we have wide format data that we can grab summary statistics with:

In [ ]:
pivoted_df.describe()

We can also provide multiple values to pivot on, which will result in a hierarchical index:

In [ ]:
pivoted_df = long_df.pivot(
    index='date', columns='datatype', values=['temp_C', 'temp_F']
)
pivoted_df.head()

With the hierarchical index, if we want to select `TMIN` in Fahrenheit, we will first need to select `temp_F` and then `TMIN`:

In [ ]:
pivoted_df['temp_F']['TMIN'].head()

#### `unstack()`

We have been working with a single index throughout this chapter; however, we can create an index from any number of columns with `set_index()`.
This gives us an index of type `MultiIndex`, where the outermost level corresponds to the first element in the list provided to `set_index()`:

In [ ]:
multi_index_df = long_df.set_index(['date', 'datatype'])
multi_index_df.head().index

Notice there are now 2 index sections of the dataframe:

In [ ]:
multi_index_df.head()

With an index of type `MultiIndex`, we can no longer use `pivot()`. We must now use `unstack()`, which by default moves the innermost index onto the columns:

In [ ]:
unstacked_df = multi_index_df.unstack()
unstacked_df.head()

The `unstack()` method also provides the `fill_value` parameter, which let's us fill-in any `NaN` values that might arise from this restructuring of the data. Consider the case that we have data for the average temperature on October 1, 2018, but no other date:

In [ ]:
extra_data = long_df.append([{
    'datatype': 'TAVG', 
    'date': '2018-10-01', 
    'temp_C': 10, 
    'temp_F': 50
}]).set_index(['date', 'datatype']).sort_index()

extra_data['2018-10-01':'2018-10-02']

If we use `unstack()` in this case, we will have `NaN` for the `TAVG` columns every day but October 1, 2018:

In [ ]:
extra_data.unstack().head()

To address this, we can pass in an appropriate `fill_value`. However, we are restricted to passing in a value for this, not a strategy (like we saw with `fillna()`), so while `-40` is definitely not be the best value, we can use it to illustrate how this works, since this is the temperature at which Fahrenheit and Celsius are equal:

In [ ]:
extra_data.unstack(fill_value=-40).head()

### Melting

Going from wide to long format.


In [ ]:
wide_df = pd.read_csv('data/02/wide_data.csv')
wide_df.head()

#### `melt()`
In order to go from wide format to long format, we use the `melt()` method. We have to specify:
- `id_vars`: which column(s) uniquely identify a row in the wide format (`date`, here)
- `value_vars`: the column(s) that contain(s) the values (`TMAX`, `TMIN`, and `TOBS`, here)

Optionally, we can also provide:
- `value_name`: what to call the column that will contain all the values once melted
- `var_name`: what to call the column that will contain the names of the variables being measured

In [ ]:
melted_df = wide_df.melt(
    id_vars='date',
    value_vars=['TMAX', 'TMIN', 'TOBS'],
    value_name='temp_C',
    var_name='measurement'
)
melted_df.head()

#### `stack()`
Another option is `stack()`, which will pivot the columns of the dataframe into the innermost level of the index (resulting in an index of type `MultiIndex`). To illustrate this, let's set our index to be the `date` column:

In [ ]:
wide_df.set_index('date', inplace=True)
wide_df.head()

By running `stack()` now, we will create a second level in our index which will contain the column names of our dataframe (`TMAX`, `TMIN`, `TOBS`). This will leave us with a `Series` object containing the values:

In [ ]:
stacked_series = wide_df.stack()
stacked_series.head()

We can use the `to_frame()` method on our `Series` object to turn it into a `DataFrame` object. Since the series doesn't have a name at the moment, we will pass in the name as an argument:

In [ ]:
stacked_df = stacked_series.to_frame('values')
stacked_df.head()

Once again, we have an index of type `MultiIndex`:

In [ ]:
stacked_df.head().index

Unfortunately, we don't have a name for the `datatype` level:

In [ ]:
stacked_df.index.names

We can use `set_names()` to address this though:

In [ ]:
stacked_df.index.set_names(['date', 'datatype'], inplace=True)
stacked_df.index.names

## Exercises

We want to look at data for the FAANG stocks (Facebook, Apple, Amazon, Netflix, and Google), but we were given each as a separate CSV file.

In [ ]:
%reload_ext tutorial.tests.testsuite

**Exercise 1**


1. Read each file in.
2. Add a column to each dataframe indicating the ticker it is for.
3. Append them together into a single dataframe.
4. Save the result to a CSV file named `faang.csv` in the folder `data/02/exercises`.


**Exercise 2**

With `faang`, use type conversion to cast the values of the date column into
datetimes and the `volume` column into integers. Then, sort by `date` and `ticker`.

**Exercise 3**

Find the seven rows in `faang` with the lowest value for `volume`.

# Data aggregation